<a href="https://colab.research.google.com/github/alinejbarbosa-web/Dev-Agentes-de-IA-Google/blob/main/Agente_de_IA_Alura_09_09_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AULA 01

In [ ]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


IMPORTAR API KEY

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

Google_API_Key = userdata.get('Gemini_Api_Key')

CONECTAR COM GEMINI

In [ ]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=Google_API_Key,
)


COLOCAR A CÉLULA PARA FUNCIONAR SOLICITANDO UMA RESPOSTA

In [ ]:
resp_test= ASBK.invoke ("Quem é você?")

INVOKE PARA INVOCAR O MODELO QUE FIZEMOS ANTERIORMENTE (ALGO ACONTECEU APÓS EXECUTAR A CÉLULA, E DEPOIS DO PRINT SABEREMOS O QUE FOI FEITO)

In [ ]:
print(resp_test.content)

In [ ]:
print(resp_test)

SEM O .CONTENT ELE DÁ TUDO QUE PODERIA TER DE VARIÁVEL RELACIONADO A COMANDO, E COM O CONTENT É MAIS RESUMIDO

In [ ]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,
    api_key=Google_API_Key
)

A TEMPERATURA FOI PARA 1.0 PARA DAR MAIS DETALHES NAS INFORMAÇÕES.

In [ ]:
resp_test= ASBK.invoke ("Quem é você? Seja criativo")
print(resp_test.content)

MANTER TEMPERATURA 0.0 PARA SEGUIR A IMERSÃO

In [ ]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=Google_API_Key
)

In [ ]:
resp_test= ASBK.invoke ("Quem é você? Seja criativo")
print(resp_test.content)

AULA 02

Instalar Bibliotecas
Community tem várias integrações com recursos.
Faiss faz busca semântica em que entende o significado e a intenção da pesquis.
Langchain-text-splitters quebra o texto em vários pedaços.
Pymupdf lê PDF.


In [ ]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

Trazer os documentos da Empresa para o Colab fazendo upload na lasta ao lado deste comando. Toda vez que precisa, terá que fazer ipload, mas pode usar o Codigo da célula para pedir ao Gemini um comando que gere o upload automaticamente do Drive todas que vez que acessar o Colab.

As bibliotecas foram instaladas no primeiro comando da AULA 02, agora serão importadas. Documentos serão lidos. Então, precisa salvar os docs que foram baixados. Cria a lista vazia para depois inserir os docs.

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Carregado com sucesso arquivo {n.name}")
    except Exception as e:
        print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [ ]:
chunks

In [ ]:
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=Google_API_Key
)

Tipo de busca e o parâmetro de busca (Threshold)

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(ASBK, prompt_rag)

In [ ]:
# Formatadores
import re, pathlib
from typing import List, Dict

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela // 2), min(len(txt), pos + janela // 2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [ ]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")